# Tutorial 00: P1 Finite Elements for the Poisson Equation

## Problem Formulation

In this exercise, we will solve our first PDE, the *Poisson equation*:


\begin{equation}
\begin{aligned}
-\Delta u & = f && \text{in}\ \Omega \\
u & = g && \text{on}\ \partial\Omega \label{eq:1a}
\end{aligned}
\end{equation}


where $\Omega \in \mathbb{R}^d$ is a given, polyhedral domain
(elements with curved boundaries are possible in DUNE but will not be considered
here). This problem is one of the basic equations of mathematical physics
which describes gravitational and electric potential as well as stationary heat or
groundwater flow. Poisson's equation is an instance of an *elliptic partial differential equation*.

A function $u \in C^2(\Omega) \cap C^0(\bar{\Omega})$ (the space of twice continuously differentiable functions in $\Omega$ which are continous up to the boundary) is called a *strong solution* if it satisfies the above equations pointwise.

Formally, one often reduces the equations to a problem with homogeneous Dirichlet boundary conditions g ≡ 0 as the starting point for theoretical considerations. We will deliberately not
do this here as it is also not done in the computer implementation of the method.

### Weak Formulation

Existence, uniqueness and stability of solutions, i.e. well-posedness in the sense of
Hadamard, is easier to prove for so-called weak solutions. As the weak formulation
is also the basis of the finite element method we explain it here.

As a start, suppose $u$ is a strong solution of the equations above and take 
*any* function
$v\in C^1(\Omega)\cap C^0(\bar\Omega)$ with $v=0$ on $\partial\Omega$, then we
have by integration by parts:
\begin{equation*}
\int_\Omega (-\Delta u) v \,dx = 
\int_\Omega \nabla u \cdot \nabla v \,dx= \int_\Omega fv \,dx.
\end{equation*}
Observe that the boundary integral $\int_{\partial\Omega} \nabla u\cdot n v\,dx$ vanishes
due to the fact that $v=0$ on $\partial\Omega$. Loosely speaking $v=0$ is a consequence
of the Dirichlet boundary condition $u=g$ on $\partial\Omega$.

Introducing the abbreviations
\begin{equation*}
a(u,v) = \int_\Omega \nabla u \cdot \nabla v \,dx, \qquad l(v) = \int_\Omega fv \,dx
\end{equation*}
one can on the other hand ask the question: Is there a class,
more specific a vector space, of functions $V$ with $V_g=\{v\in V : 
\text{$v=g$ on $\partial\Omega$}\}$ and $V_0=\{v\in V : 
\text{$v=0$ on $\partial\Omega$}\}$ such that
the problem
\begin{equation} 
u \in V_g :\qquad a(u,v) = l(v) \qquad \forall v\in V_0 \label{eq:weakform}
\end{equation}
has a unique solution.

The answer is yes and in particular one can prove the following:

1. With $V=H^1(\Omega)$, the Sobolev space of functions with square integrable
weak derivatives, the problem \eqref{eq:weakform} has a unique solution provided
the bilinear form $a : V\times V \to \mathbb{R}$ is continuous and coercive on the 
subspace $V_0\subset V$ and the linear form $l: V \to \mathbb{R}$ is also continuous. 
Coercivity on $V_0$ follows from Friedrich's inequality and for the continuity of the right hand
side functional $f\in L^2(\Omega)$ is a sufficient condition.
2. If in addition $u\in C^2(\Omega)\cap C^0(\bar\Omega)$, then the solutions
of \eqref{eq:weakform} and \eqref{eq:1a} coincide.

We call \eqref{eq:weakform} the *weak formulation* of \eqref{eq:1a}.
As it has a unique solution under more general conditions than \eqref{eq:1a},
e.g. discontinuous right hand side functions $f$, it can be considered a generalization
of the problem.

## The Finite Element Method

The (conforming) finite element method, in a nutshell, is based on the weak formulation where
the function space $V$ is replaced by a subspace $V_h\subset V$ 
which is *finite dimensional*. Here, the subscript $h$ relates to the dimension of the
function space. One major part of the finite element method is to construct such so-called *finite element spaces*.
Typically, finite element spaces consist of piecewise polynomial functions.
We consider one particular choice, the space of linear functions on simplicial elements.
For a more detailed description of the underlying finite element method, please refer to `tutorial00.pdf`. Additionally, there are many text books about the finite element method, which are listed in the further reading section at the end of this tutorial.

## Implementation

The code presented in this tutorial solves the elliptic equation \eqref{eq:1a} with Dirichlet boundary conditions, where
\begin{align*}
    f(x) = -2d \quad\text{and}\quad  g(x) = \sum_{i=1}^d (x)_i^2.
\end{align*}

First, we again include all the Dune sources through one convenience header:

In [ ]:
#include<dune/jupyter.hh>

The simulation code written below will depend on a number of runtime parameters that are provided in a configuration file `tutorial00.ini`. You find that file in the `notebooks` directory. Dune uses a nested data structured called `ParameterTree` for these configurations. In the following we parse the configuration, output it to the notebook and alter it from within the notebook:

In [ ]:
Dune::ParameterTree ptree;
Dune::ParameterTreeParser::readINITree("tutorial00.ini", ptree);

In [ ]:
ptree

In [ ]:
ptree["grid.refinement"] = "0";

In [ ]:
ptree

We are using a two-dimensional, unstructured mesh for this simulation. The mesh resolves the unit square $\Omega = [0,1]^2$. The file is specified in the `unitsquare.msh` file that was generated using Gmsh.

In [ ]:
const int dim = 2;
using Grid = Dune::UGGrid<dim>;
std::string filename = ptree.get("grid.twod.filename", "unitsquare.msh");
Dune::GridFactory<Grid> factory;
Dune::GmshReader<Grid>::read(factory,filename,true,true);
std::unique_ptr<Grid> grid(factory.createGrid());
grid->globalRefine(ptree.get<int>("grid.refinement"));
auto gv = grid->leafGridView();
using GV = decltype(gv);

In [ ]:
grid

With Dune making heavy use of C++ templates, the underlying floating point type of the simulation can be chosen quite freely. Looking for a solution $u: \Omega\rightarrow\mathbb{R}$, the *domain field* type is the floating point type used to realize $\Omega$ where as the *range field* type is used to realize $\mathbb{R}$. `DF` is defined by the grid implementation (usually to `double`), but we can freely choose `RF` - typically also to `double`:

In [ ]:
using DF = Grid::ctype;
using RF = double;

In the following, first a *local finite element map* of type `PkLocalFiniteElementMap` is set up. The finite element map associates finite element basis functions, defined on the corresponding reference element, with each element of the mesh. Additionally, information is provided on how the global finite element space $V_h$ is constructed from the local, element-wise pieces.

The type `CON` provides a so-called constraints class, which provides a way to assemble constraints on a function space.

The type `VBE` provides a vector backend, this allows for the direct filling of different types of data into these vectors, without a copy step. This is usefull, as PDELab can use different iterative solver libraries, which provide their own data types for vectors and (sparse) matrices. In this code, DUNE's own iterative solver library ISTL provides the vector backend.

From the above described parameters, the type `GFS` from the class template `GridFunctionSpace`is defined. This combines all the given information to construct the global finite element space $V_h$ on a given grid view. In the last two lines an object of this class is instantiated and the space is given a name.

In [ ]:
using FEM = Dune::PDELab::PkLocalFiniteElementMap<GV,DF,RF,1>;
FEM fem(gv);
using CON = Dune::PDELab::ConformingDirichletConstraints;
using VBE = Dune::PDELab::ISTL::VectorBackend<>;
using GFS = Dune::PDELab::GridFunctionSpace<GV,FEM,CON,VBE>;
GFS gfs(gv,fem);
gfs.name("P1");

In the next step, a variable that will later on contain the solution vector *z* is declared:

In [ ]:
using Z = Dune::PDELab::Backend::Vector<GFS,RF>;
Z z(gfs);

The function g extends the Dirichlet boundary values into the inerior. This function can be used to provide e.g. the exact solution of the problem for testing purposes or an initial guess for the iterative solvers.

In [ ]:
auto g = Dune::PDELab::makeGridFunctionFromCallable(
    gv,
    [](const auto& x){
        RF s=0.0;
        for (std::size_t i=0; i<x.size(); i++)
            s+=x[i]*x[i];
        return s;
    }
);
Dune::PDELab::interpolate(g,gfs,z);

This is the solution for exercise 1.1:

In [ ]:
/*auto g = Dune::PDELab::makeGridFunctionFromCallable(
    gv,
    [](const auto& x){
        RF s=0.0;
        for (std::size_t i=0; i<x.size(); i++)
            s+=x[i]*x[i]*x[i];
        return s;
    }
);
Dune::PDELab::interpolate(g,gfs,z);
*/

In the following, the constraints for a specific grid function space are determined from the boundary condition type function `b`. They are saved into the constraints container `cc` of type `CC`. The separation of the function space $V_h$ and the constraints set $\mathcal{I}_h^{\partial \Omega}$ allows one to reuse the function space together with different constraints, e.g. for a system of partial differential equations.

In [ ]:
using CC = typename GFS::template ConstraintsContainer<RF>::Type;
CC cc;

In [ ]:
auto b = Dune::PDELab::makeBoundaryConditionFromCallable(
    gv,
    [](const auto& x){ return true; }
);
Dune::PDELab::constraints(b, gfs, cc);
std::cout << "constrained dofs=" << cc.size() << " of "
          << gfs.globalSize() << std::endl;

The right hand side $f$ of the partial differential equation is denoted by the function `f`.

In [ ]:
auto f = Dune::PDELab::makeGridFunctionFromCallable(
    gv,
    [](const auto& x){
        return Dune::FieldVector<RF,1>(-2.0*x.size());
    }
);;

This is the solution to exercise 1.1:

In [ ]:
/*auto f = Dune::PDELab::makeGridFunctionFromCallable(
    gv,
    [](const auto& x){
        RF s=0.0;
        RF t=0.0;
        for (std::size_t i=0; i<x.size(); i++){
            s+=x[i];
            t+=x[i]*x[i]*x[i];
        }
        return -6*s;
    }
);;
*/

Next, we will construct the local operator. We include the local operator implementation for P1-conforming finite elements from the file `poissonp1.hh`. You find this file in the `notebooks` folder. After making changes to the file, you need to restart and rerun this notebook.

In [ ]:
#include "poissonp1.hh"

using LOP = PoissonP1<decltype(f),FEM>;
LOP lop(f, fem.find(*gv.template begin<0>()));

This local operator is used as one of the template arguments in the global assembler or grid operator, which additionally requires the types for ansatz and test space, a matrix backend, the types for the components of the coefficient vectors of ansatz and test space and the entries of the matrix *A* and finally the types of the constraints container for test and ansatz space.


In [ ]:
using MBE = Dune::PDELab::ISTL::BCRSMatrixBackend<>;
MBE mbe(1<<(dim+1)); // guess nonzeros per row
using GO = Dune::PDELab::GridOperator<
  GFS,GFS,  /* ansatz and test space */
  LOP,      /* local operator */
  MBE,      /* matrix backend */
  RF,RF,RF, /* domain, range, jacobian field type*/
  CC,CC     /* constraints for ansatz and test space */
  >;
GO go(gfs,cc,gfs,cc,lop,mbe);

Next, a linear solver for the linear system $Az = b$ is selected. Since ISTL backends for vectors and matrices are used, also a linear solver from the ISTL library is chosen.
Complete solvers are packaged by PDELab for sequential and
parallel computations. Here we select the conjugate gradient method with algebraic
multigrid as preconditioner and symmetric successive overrelaxation as smoother in
multigrid in its sequential implementation. The linear solver object `ls` is initialized
with the maximum number of iterations and a verbose parameter:

In [ ]:
using LS = Dune::PDELab::ISTLBackend_SEQ_CG_AMG_SSOR<GO>;
LS ls(100,2);

Different linear iterative solver (exercise 1.3):

In [ ]:
// using LS = Dune::PDELab::ISTLBackend_SEQ_BCGS_SSOR;
// LS ls(5000,true);

So far, no finite element computations have actually been performed, only the
necessary components have been configured to now do the real work together. We
can now set up the matrix *A* as well as the right hand side *b* and then solve the
system. Since this is required often, there is a class in PDELab for this:

In [ ]:
using SLP = Dune::PDELab::StationaryLinearProblemSolver<GO,LS,Z>;
SLP slp(go,ls,z,1e-10);
slp.apply(); // here all the work is done!

In the given example a problem with a known exact solution, which is given by
the function *g* is solved. In order to compare the computed solution with the exact
solution we initialize another coeffcient vector with the Lagrange interpolant of the
exact solution and provide a grid function:

In [ ]:
using ZDGF = Dune::PDELab::DiscreteGridFunction<GFS,Z>;
ZDGF zdgf(gfs,z);
Z w(gfs); // Lagrange interpolation of exact solution
Dune::PDELab::interpolate(g,gfs,w);
ZDGF wdgf(gfs,w);
Dune::VTKWriter<GV> vtkwriter(gv,Dune::VTK::conforming);
using VTKF = Dune::PDELab::VTKGridFunctionAdapter<ZDGF>;
vtkwriter.addVertexData(std::shared_ptr<VTKF>(new VTKF(zdgf,"fesol")));
vtkwriter.addVertexData(std::shared_ptr<VTKF>(new VTKF(wdgf,"exact")));

The visualization data can be generated in Jupyter by printing the VTKWriter instance:

In [ ]:
vtkwriter

# Exercises

Some comment on how to work through the exercises 

## Exercise 1

1. **Warming up**

   Run the program with different refinement levels. Visualize the solution in ParaView, use the Calculator
   filter to visualize the difference $|u-u_h|$ and determine the
   maximum error. Note that $u$ and $u_h$ are called `exact` and `fesol` in the paraview output.

2. **Solving a new problem**

   Now consider
   \begin{equation}
   f(x) = -\sum_{i = 1}^{d} 6(x)_i \quad \text{and} \quad g(x) = \sum_{i = 1}^d(x)_i^2
   \end{equation}
   and check that $u(x)=\sum_{i=1}^d (x)_i^3$ solves the PDE. Implement the new $f$ and $g$ and rerun your program.

3. **Analysis of finite element error**:

  Produce a sequence of output files for different levels of mesh
  refinements $0, 1, 2, \ldots$ with suitable output
  filenames. Visualize the error $|u-u_h|$ in Para\-View and determine
  the maximum error on each level.  If you want you can also try to
  calculate the L2-norm of the error in paraview and determine the
  convergence rate by hand. In a later exercise you will see how to
  calculate the L2-norm directly in the c++ source.

## solution

4. **Use a different solver**

   You may also try to exchange the iterative linear solver, the lines for the new solver are already given as solution next to the old linear solver. Compare the number of iterations which is given by the following lines
  in the output (here we have 12 iterations):


=== CGSolver

12      1.9016e-09

=== rate=0.144679, T=0.02362, TIT=0.00196833, IT=12

## solution

## Exercise 2

# Further Reading

Finite element method:
1. K. Eriksson, D. Estep, P. Hansbo and C. Johnson. *Computational Differential Equations*. Cambridge University Press.1996. 
2. A. Ern and J.-L. Guermond. *Theory and practice of finite element methods*. Springer. 2004.
3. P. G. Ciarlet. *The finite element method for elliptic problems*. SIAM. Classics in Applied Mathematics. 2002.
4. D. Braess. *Finite Elemente*. Springer. 2003.
5. S. C. Brenner and L. R. Scott. *The mathematical theory of finite element methods*. Springer. 1994.
6. H. Elman, D. Silvester and A. Wathen.*Finite Elements and Fast Iterative Solvers*. Oxford University Press. 2005.
7. C. Großmann and H.-G. Roos. *Numerische Behandlung partieller Differentialgleichungen*. Teubner. 2006.
8. W. Hackbusch. *Theorie und Numerik elliptischer Differentialgleichungen*. Teubner. 1986.
9. R. Rannacher. *Einführung in die Numerische Mathematik II (Numerik partieller Differentialgleichungen)*. 2006. http://numerik.iwr.uni-heidelberg.de/~lehre/notes.
10. P. Bastian. *Lecture Notes on Scientific Computing with Partial Differential Equations*. 2014. https://conan.iwr.uni-heidelberg.de/data/teaching/finiteelements_ws2017/num2.pdf.